In [2]:
import mlflow.xgboost
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

In [1]:
experiment_name = "Alzheimer_Preprocesamiento"
run_id = "de170f0e0b5748888f38e1b70e6b7577"

# --- Transformer load and model---
prep_path = mlflow.artifacts.download_artifacts(
    run_id=run_id,
    artifact_path="preprocessor/preprocessor.pkl"
)

with open(prep_path, "rb") as f:
    transformador = pickle.load(f)

modelo = mlflow.xgboost.load_model(
    model_uri="models:/Alzheimer_XGBoost/3"
)

In [ ]:
# X_new_raw, y_new_raw → nuevos pacientes
X_new_prep = transformador.transform(X_new_raw)
y_new = y_new_raw

In [ ]:
from xgboost import XGBClassifier

modelo_retrain = XGBClassifier(
    n_estimators=100,          # SOLO árboles nuevos
    learning_rate=0.05,
    max_depth=5,
    eval_metric="logloss",
    random_state=42
)

modelo_retrain.fit(
    X_new_prep,
    y_new,
    xgb_model=modelo_base      # ← continúa desde v1
)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

y_pred = modelo_retrain.predict(X_test)

acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy (retrain):", acc)
print("F1 (retrain):", f1)